In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import datetime
import math
import numpy
import os
import pandas
import pathlib
import skimage.io
import skimage.transform
import sklearn.model_selection
import tensorflow
import time


from files import create_folder, save_fit_history, save_lossgraph, save_figs
from metrics import dice_coef, jaccard_distance
from model import evaluate, unet_model, get_loss_function

In [ ]:
class CreateSequence(tensorflow.keras.utils.Sequence):
    def __init__(self, batch_size, img_size, input_img_paths, target_img_paths):
        self.batch_size = batch_size
        self.img_size = img_size
        self.input_img_paths = input_img_paths
        self.target_img_paths = target_img_paths

    def __len__(self):
        return len(self.target_img_paths) // self.batch_size

    def __getitem__(self, idx):
        """Returns tuple (input, target) correspond to batch #idx."""
        i = idx * self.batch_size
        batch_input_img_paths = self.input_img_paths[i : i + self.batch_size]
        batch_target_img_paths = self.target_img_paths[i : i + self.batch_size]

        x = numpy.zeros((self.batch_size,) + self.img_size + (3,), dtype="float32")
        for j, path in enumerate(batch_input_img_paths):
            img = skimage.io.imread(path.resolve())
            img = numpy.float32(img/255)
            x[j] = img

        y = numpy.zeros((self.batch_size,) + self.img_size + (1,), dtype="float32")
        for j, path in enumerate(batch_target_img_paths):
            mask = skimage.io.imread(path.resolve())
            mask = numpy.expand_dims(mask, 2)
            mask = numpy.float32(mask/255)
            y[j] = mask

        return x, y

# GPU

In [ ]:
gpus = tensorflow.config.experimental.list_physical_devices("GPU")
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            print(f"GPU: {gpu.name}")
            tensorflow.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

In [ ]:
cfg = {
    "channel": 3,
    "batch_size": 4,
    "fold": 5,
    "epochs": 75,
    "image_size": 256,
    "learning_rate": 0.001,
    "random_state": 1234,
    "test_size": 0.2,
    "val_size": 0.05,
    "path_dataset": "dataset",
    "path_out": "out",
    "loss_function": "dice",
    "data_augmentation": False
}
images_folder = os.path.join(cfg["path_dataset"], "IMAGEM_ORIGINAL", "CONVERTIDAS", "RGB", "256", "OUT")
masks_folder = os.path.join(cfg["path_dataset"], "MASK", "BITMAP", "256", "OUT")

In [ ]:
x = numpy.array(list([file for file in sorted(pathlib.Path(images_folder).rglob("*")) if file.is_file()]))
y = numpy.array(list([file for file in sorted(pathlib.Path(masks_folder).rglob("*")) if file.is_file()]))

len(x), len(y)

In [ ]:
data = CreateSequence(cfg["batch_size"], (cfg["image_size"], cfg["image_size"]), x, y)
for d in data:
    print(d)

In [ ]:
kf = sklearn.model_selection.KFold(n_splits=cfg["fold"], shuffle=True, random_state=cfg["random_state"])
models = []
list_evaluate = list([])
current_datetime = datetime.datetime.now().strftime("%d-%m-%Y-%H-%M-%S")
path = os.path.join(cfg["path_out"], current_datetime)
create_folder(list([path]))
for fold, (train_index, test_index) in enumerate(kf.split(x)):
    x_train, x_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]
    x_train, x_val, y_train, y_val = sklearn.model_selection.train_test_split(x_train, y_train, test_size=cfg["val_size"], random_state=cfg["random_state"])

    train_data = CreateSequence(cfg["batch_size"], (cfg["image_size"], cfg["image_size"]), x_train, y_train)
    val_data = CreateSequence(cfg["batch_size"], (cfg["image_size"], cfg["image_size"]), x_val, y_val)
    test_data = CreateSequence(cfg["batch_size"], (cfg["image_size"], cfg["image_size"]), x_test, y_test)

    print(x_train.shape)
    print(x_val.shape)
    print(x_test.shape)
    print(x.shape)

    path_fold = os.path.join(path, str(fold))
    create_folder(list([path_fold]))

    steps_per_epoch = math.ceil(x_train.shape[0] / cfg["batch_size"])
    reduce_learning_rate = tensorflow.keras.callbacks.ReduceLROnPlateau(monitor="loss", factor=0.5, patience=3, verbose=1)
    filename_model = os.path.join(path_fold, "unet.h5")
    checkpointer = tensorflow.keras.callbacks.ModelCheckpoint(filename_model, verbose=1, save_best_only=True)
    strategy = tensorflow.distribute.MirroredStrategy()

    with strategy.scope():
        model = unet_model(cfg)
        adam_opt = tensorflow.keras.optimizers.Adam(learning_rate=cfg["learning_rate"])
        model.compile(optimizer=adam_opt, loss=get_loss_function(cfg["loss_function"]), metrics=[dice_coef, jaccard_distance, tensorflow.keras.metrics.Precision(), tensorflow.keras.metrics.Recall()])

    tensorflow.keras.backend.clear_session()
    start_time = time.time()
    fit = model.fit(train_data, steps_per_epoch=steps_per_epoch, epochs=cfg["epochs"], validation_data=val_data, callbacks=[checkpointer, reduce_learning_rate])
    end_time = time.time() - start_time

    save_fit_history(fold, fit, path_fold)
    save_lossgraph(fold, fit, path_fold)
    # evaluate(train_data)
    # list_evaluate.append(evaluate(end_time, fold, model, x_train, x_val, x_test, y_train, y_val, y_test))

    # models.append(model)

    # model = tensorflow.keras.models.load_model("unet_rgb.h5", custom_objects = {"dice_loss": dice_loss, "dice_coef": dice_coef, "jaccard_distance": jaccard_distance })

    # save_figs(cfg, list_images_names, test_index, model, path_fold, x)
tensorflow.keras.backend.clear_session()